# Libraries

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import random as rd
import pandas as pd
import numpy as np
import pickle as pkl
from tqdm import tqdm
import os
import time
from collections import defaultdict
from scipy import stats
from itertools import permutations, product
from numpy.linalg import norm
import seaborn as sns
sns.set(style="darkgrid")
pd.set_option("display.precision", 4)

# Summary

#### Evaluation strategies
We have identified three evaluation strategies:
1. For every user in the **train set** rank **all items**. | Book paper, *eva_one*
2. For every user in the **test set** rank **only the items this user has rated in the test set**. | Music paper, *eva_two*
3. For every user in the **test set** rank **only the items this user has NOT rated in the train set**. | Movie(?) paper, *eva_three*

#### Algorithms

In [ ]:
algo_names = ["UserKNN",
              "ItemKNN", 
              "UserKNN with means", 
              "BPR",
              "MF",
              "PMF",
              "NMF",
              "WMF",
              "HPF",
              "NeuMF",
              "VAECF"
              ] # in the order that they are in the model list

# Import results

### Additions

In [ ]:
data = ["movies", "music","books"]
evaluation = ["eva_one", "eva_two", "eva_three"]
pop_notion = ["pop_one", "pop_two", "pop_three"]

In [ ]:
aspects = [data, evaluation, pop_notion]

In [ ]:
some_aspects = [data, evaluation]

In [ ]:
results_location = "data/results/"

In [ ]:
# pkl.load(open(results_location+"df_item_dist_cornac_"+data[2]+"_"+evaluation[0]+".csv","rb"))

In [ ]:
c = pd.read_csv(results_location+"df_item_dist_cornac_"+data[2]+"_"+evaluation[0]+".csv", index_col=0)

In [ ]:
c2 = pd.read_csv(results_location+"df_item_dist_cornac_"+data[2]+"_"+evaluation[1]+".csv", index_col=0)

### Define aspect combinations

In [ ]:
all_combinations = list(product(*aspects))
all_combinations = [x for x in all_combinations if not ("movies" in x)&("pop_three" in x)]
all_combinations = [x for x in all_combinations if not ("books" in x)&("pop_three" in x)]
# remove combinations of "pop_three" that dont include music.
some_combinations = list(product(*some_aspects))

In [ ]:
sum_exists = 0
for combi in some_combinations:
    path = results_location+"df_item_dist_cornac_"+combi[0]+"_"+combi[1]+".csv"
    exists = os.path.exists(path)
    print(combi, exists)
    sum_exists+=exists
print(sum_exists, "out of", len(some_combinations))

In [ ]:
new_results_location = 'results/'


In [ ]:
sum_exists = 0

for combi in all_combinations:
    path = new_results_location+"low_gap_vals_"+combi[0]+"_"+combi[1]+"_"+combi[2]+".pickle"
    exists = os.path.exists(path)
    sum_exists+=exists
    print(combi, "|",exists)
print(sum_exists, "out of", len(all_combinations))

# GAP

## TTESTS

In [ ]:
music_GAPs_eva1

In [ ]:
new_results_location = 'results/'
for combi in all_combinations:
    
    if 'music' in combi: 
        eva_str = combi[1]
        pop_n = combi[2]
        print(eva_str, pop_n)
        print('----------------')
        #path_ttest = 'gap_ttests_books_eva_one_pop_one.pickle'
        path_ttest = new_results_location+"gap_ttests_music_"+eva_str+"_"+pop_n+".pickle"
        pvalues = pd.DataFrame(pkl.load(open(path_ttest,"rb") )[2:], index=algo_names, columns = ['low-med','low-high','med-high'])
        print(pvalues)
        print((pvalues<0.005) & (pvalues>0.0) & (pvalues<1.0))

In [ ]:
combi = all_combinations[0]
path = new_results_location+"low_gap_vals_"+combi[0]+"_"+combi[1]+"_"+combi[2]+".pickle"


In [ ]:
pop_gaps = []
for p in pop_notion:
    for v in ["low","med","high"]:
        pop_gaps.append(p+"_"+v)

In [ ]:
pop_gaps

### Music

In [ ]:
music_GAPs_eva1 = pd.DataFrame(columns=pop_gaps, index = algo_names).fillna(0)
music_GAPs_eva2 = pd.DataFrame(columns=pop_gaps, index = algo_names).fillna(0)
music_GAPs_eva3 = pd.DataFrame(columns=pop_gaps, index = algo_names).fillna(0)

In [ ]:
for combi in all_combinations:
    if "music" in combi:
        eva_str = combi[1]
        pop_n = combi[2]
        path_low = results_location+"low_gap_vals_music_"+eva_str+"_"+pop_n+".pickle"
        path_med = results_location+"med_gap_vals_music_"+eva_str+"_"+pop_n+".pickle"
        path_high = results_location+"high_gap_vals_music_"+eva_str+"_"+pop_n+".pickle"
        low_gap_vals = pd.DataFrame([pkl.load(open(path_low,"rb") )[2:]], columns=algo_names)
        med_gap_vals = pd.DataFrame([pkl.load(open(path_med,"rb") )[2:]], columns=algo_names)
        high_gap_vals = pd.DataFrame([pkl.load(open(path_high,"rb") )[2:]], columns=algo_names)
        
        if eva_str=="eva_one":
            for algorithm in algo_names:
                music_GAPs_eva1.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                music_GAPs_eva1.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                music_GAPs_eva1.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
                        
        elif eva_str=="eva_two":
            for algorithm in algo_names:
                music_GAPs_eva2.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                music_GAPs_eva2.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                music_GAPs_eva2.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
            
        else:
            for algorithm in algo_names:
                music_GAPs_eva3.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                music_GAPs_eva3.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                music_GAPs_eva3.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]

In [ ]:
np.round(music_GAPs_eva3,1)

In [ ]:
music_GAPs_eva1.columns =["eva_one_"+x for x in music_GAPs_eva1.columns]
music_GAPs_eva2.columns =["eva_two_"+x for x in music_GAPs_eva2.columns]
music_GAPs_eva3.columns =["eva_three_"+x for x in music_GAPs_eva3.columns]

In [ ]:
combined_music_GAPs = pd.concat([music_GAPs_eva1, music_GAPs_eva2, music_GAPs_eva3], axis=1)

In [ ]:
combined_music_GAPs = np.round(combined_music_GAPs,1)

In [ ]:
with open("latex_tables/music_GAPs_combined.tex", "w") as tf:
    tf.write(combined_music_GAPs.to_latex())

In [ ]:
with open("latex_tables/music_GAPs_eva1.tex", "w") as tf:
    tf.write(np.round(music_GAPs_eva1,1).to_latex())
with open("latex_tables/music_GAPs_eva2.tex", "w") as tf:
    tf.write(np.round(music_GAPs_eva2,1).to_latex())
with open("latex_tables/music_GAPs_eva3.tex", "w") as tf:
    tf.write(np.round(music_GAPs_eva3,1).to_latex())

### Books

#### ttest

In [ ]:
new_results_location = 'results/'
for combi in all_combinations:
    
    if 'books' in combi: 
        eva_str = combi[1]
        pop_n = combi[2]
        print(eva_str, pop_n)
        print('----------------')
        #path_ttest = 'gap_ttests_books_eva_one_pop_one.pickle'
        path_ttest = new_results_location+"gap_ttests_books_"+eva_str+"_"+pop_n+".pickle"
        pvalues = pd.DataFrame(pkl.load(open(path_ttest,"rb") )[2:], index=algo_names, columns = ['low-med','low-high','med-high'])
        print(pvalues)

#### gap

In [ ]:
books_GAPs_eva1 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
books_GAPs_eva2 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
books_GAPs_eva3 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
for combi in all_combinations:
    if "books" in combi:
        eva_str = combi[1]
        pop_n = combi[2]
        path_low = new_results_location+"low_gap_vals_books_"+eva_str+"_"+pop_n+".pickle"
        path_med = new_results_location+"med_gap_vals_books_"+eva_str+"_"+pop_n+".pickle"
        path_high = new_results_location+"high_gap_vals_books_"+eva_str+"_"+pop_n+".pickle"
        low_gap_vals = pd.DataFrame([pkl.load(open(path_low,"rb") )[2:]], columns=algo_names)
        med_gap_vals = pd.DataFrame([pkl.load(open(path_med,"rb") )[2:]], columns=algo_names)
        high_gap_vals = pd.DataFrame([pkl.load(open(path_high,"rb") )[2:]], columns=algo_names)
        
        if eva_str=="eva_one":
            for algorithm in algo_names:
                books_GAPs_eva1.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                books_GAPs_eva1.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                books_GAPs_eva1.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
                        
        elif eva_str=="eva_two":
            for algorithm in algo_names:
                books_GAPs_eva2.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                books_GAPs_eva2.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                books_GAPs_eva2.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
            
        else:
            for algorithm in algo_names:
                books_GAPs_eva3.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                books_GAPs_eva3.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                books_GAPs_eva3.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]

In [ ]:
books_GAPs_eva1

In [ ]:
books_GAPs_eva1.columns =["eva_one_"+x for x in books_GAPs_eva1.columns]
books_GAPs_eva2.columns =["eva_two_"+x for x in books_GAPs_eva2.columns]
books_GAPs_eva3.columns =["eva_three_"+x for x in books_GAPs_eva3.columns]

In [ ]:
books_GAPs_eva1

In [ ]:
print(np.round(books_GAPs_eva1,1).to_latex())

In [ ]:
with open("latex_tables/books_GAPs_eva1.tex", "w") as tf:
    tf.write(np.round(books_GAPs_eva1,1).to_latex())
with open("latex_tables/books_GAPs_eva2.tex", "w") as tf:
    tf.write(np.round(books_GAPs_eva2,1).to_latex())
with open("latex_tables/books_GAPs_eva3.tex", "w") as tf:
    tf.write(np.round(books_GAPs_eva3,1).to_latex())

### Movies

#### ttests

In [ ]:
for combi in all_combinations:
    
    if 'movies' in combi: 
        eva_str = combi[1]
        pop_n = combi[2]
        print(eva_str, pop_n)
        print('----------------')
        #path_ttest = 'gap_ttests_books_eva_one_pop_one.pickle'
        path_ttest = new_results_location+"gap_ttests_books_"+eva_str+"_"+pop_n+".pickle"
        print(pd.DataFrame(pkl.load(open(path_ttest,"rb") )))
        # pvalues = pd.DataFrame(pkl.load(open(path_ttest,"rb") ), index=algo_names, columns = ['low-med','low-high','med-high'])
        # print(pvalues)
        break

In [ ]:
new_results_location = 'results/'
for combi in all_combinations:
    
    if 'movies' in combi: 
        eva_str = combi[1]
        pop_n = combi[2]
        print(eva_str, pop_n)
        print('----------------')
        #path_ttest = 'gap_ttests_books_eva_one_pop_one.pickle'
        path_ttest = new_results_location+"gap_ttests_books_"+eva_str+"_"+pop_n+".pickle"
        pvalues = pd.DataFrame(pkl.load(open(path_ttest,"rb") )[2:], index=algo_names, columns = ['low-med','low-high','med-high'])
        print(pvalues)
        

#### gaps

In [ ]:
movies_GAPs_eva1 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
movies_GAPs_eva2 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
movies_GAPs_eva3 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
for combi in all_combinations:
    if "movies" in combi:
        eva_str = combi[1]
        pop_n = combi[2]
        path_low = new_results_location+"low_gap_vals_movies_"+eva_str+"_"+pop_n+".pickle"
        path_med = new_results_location+"med_gap_vals_movies_"+eva_str+"_"+pop_n+".pickle"
        path_high = new_results_location+"high_gap_vals_movies_"+eva_str+"_"+pop_n+".pickle"
        low_gap_vals = pd.DataFrame([pkl.load(open(path_low,"rb") )[2:]], columns=algo_names)
        med_gap_vals = pd.DataFrame([pkl.load(open(path_med,"rb") )[2:]], columns=algo_names)
        high_gap_vals = pd.DataFrame([pkl.load(open(path_high,"rb") )[2:]], columns=algo_names)
        
        if eva_str=="eva_one":
            for algorithm in algo_names:
                movies_GAPs_eva1.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                movies_GAPs_eva1.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                movies_GAPs_eva1.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
                        
        elif eva_str=="eva_two":
            for algorithm in algo_names:
                movies_GAPs_eva2.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                movies_GAPs_eva2.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                movies_GAPs_eva2.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
            
        else:
            for algorithm in algo_names:
                movies_GAPs_eva3.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                movies_GAPs_eva3.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                movies_GAPs_eva3.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]

In [ ]:
movies_GAPs_eva1.columns =["eva_one_"+x for x in movies_GAPs_eva1.columns]
movies_GAPs_eva2.columns =["eva_two_"+x for x in movies_GAPs_eva2.columns]
movies_GAPs_eva3.columns =["eva_three_"+x for x in movies_GAPs_eva3.columns]

In [ ]:
with open("latex_tables/movies_GAPs_eva1.tex", "w") as tf:
    tf.write(np.round(movies_GAPs_eva1,1).to_latex())
with open("latex_tables/movies_GAPs_eva2.tex", "w") as tf:
    tf.write(np.round(movies_GAPs_eva2,1).to_latex())
with open("latex_tables/movies_GAPs_eva3.tex", "w") as tf:
    tf.write(np.round(movies_GAPs_eva3,1).to_latex())

## NDCGs

### Music

In [ ]:
music_NDCGs_eva1 = pd.DataFrame(columns=pop_gaps, index = algo_names).fillna(0)
music_NDCGs_eva2 = pd.DataFrame(columns=pop_gaps, index = algo_names).fillna(0)
music_NDCGs_eva3 = pd.DataFrame(columns=pop_gaps, index = algo_names).fillna(0)

In [ ]:
for combi in all_combinations:
    if "music" in combi:
        eva_str = combi[1]
        pop_n = combi[2]
        #print(eva_str, pop_n)
        path_ttest = new_results_location+"NDCGs_music_"+eva_str+"_"+pop_n+".csv"
        ndcg_df = pd.read_csv(path_ttest, index_col=0)
        #print(ndcg_df)
        
        if eva_str=="eva_one":
            for algorithm in algo_names:
                music_NDCGs_eva1.at[algorithm, pop_n+"_low"] = ndcg_df.loc[algorithm]['low']
                music_NDCGs_eva1.at[algorithm, pop_n+"_med"] = ndcg_df.loc[algorithm]['med']
                music_NDCGs_eva1.at[algorithm, pop_n+"_high"] =ndcg_df.loc[algorithm]['high']
                        
        elif eva_str=="eva_two":
            for algorithm in algo_names:
                music_NDCGs_eva2.at[algorithm, pop_n+"_low"] = ndcg_df.loc[algorithm]['low']
                music_NDCGs_eva2.at[algorithm, pop_n+"_med"] = ndcg_df.loc[algorithm]['med']
                music_NDCGs_eva2.at[algorithm, pop_n+"_high"] =ndcg_df.loc[algorithm]['high']
            
        else:
            for algorithm in algo_names:
                music_NDCGs_eva3.at[algorithm, pop_n+"_low"] = ndcg_df.loc[algorithm]['low']
                music_NDCGs_eva3.at[algorithm, pop_n+"_med"] = ndcg_df.loc[algorithm]['med']
                music_NDCGs_eva3.at[algorithm, pop_n+"_high"] =ndcg_df.loc[algorithm]['high']

In [ ]:
with open("latex_tables/music_NDCGs_eva1.tex", "w") as tf:
    tf.write(np.round(music_NDCGs_eva1,3).to_latex())
with open("latex_tables/music_NDCGs_eva2.tex", "w") as tf:
    tf.write(np.round(music_NDCGs_eva2,3).to_latex())
with open("latex_tables/music_NDCGs_eva3.tex", "w") as tf:
    tf.write(np.round(music_NDCGs_eva3,3).to_latex())

#### ttest

In [ ]:
new_results_location = 'results/'
for combi in all_combinations:
    
    if 'music' in combi: 
        eva_str = combi[1]
        pop_n = combi[2]
        print(eva_str, pop_n)
        print('----------------')
        path_ttest = new_results_location+"NDCG_ttests_music_"+eva_str+"_"+pop_n+".csv"
        pvalues = pd.read_csv(path_ttest, index_col=0).drop(['Random','MostPop'])
        print((pvalues<0.005)&(pvalues>0.0)&(pvalues!=1.0))

### Books

In [ ]:
books_NDCGs_eva1 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
books_NDCGs_eva2 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
books_NDCGs_eva3 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)

In [ ]:
for combi in all_combinations:
    if "books" in combi:
        eva_str = combi[1]
        pop_n = combi[2]
        #print(eva_str, pop_n)
        path_ttest = new_results_location+"NDCGs_books_"+eva_str+"_"+pop_n+".csv"
        ndcg_df = pd.read_csv(path_ttest, index_col=0)
        #print(ndcg_df)
        
        if eva_str=="eva_one":
            for algorithm in algo_names:
                books_NDCGs_eva1.at[algorithm, pop_n+"_low"] = ndcg_df.loc[algorithm]['low']
                books_NDCGs_eva1.at[algorithm, pop_n+"_med"] = ndcg_df.loc[algorithm]['med']
                books_NDCGs_eva1.at[algorithm, pop_n+"_high"] =ndcg_df.loc[algorithm]['high']
                        
        elif eva_str=="eva_two":
            for algorithm in algo_names:
                books_NDCGs_eva2.at[algorithm, pop_n+"_low"] = ndcg_df.loc[algorithm]['low']
                books_NDCGs_eva2.at[algorithm, pop_n+"_med"] = ndcg_df.loc[algorithm]['med']
                books_NDCGs_eva2.at[algorithm, pop_n+"_high"] =ndcg_df.loc[algorithm]['high']
            
        else:
            for algorithm in algo_names:
                books_NDCGs_eva3.at[algorithm, pop_n+"_low"] = ndcg_df.loc[algorithm]['low']
                books_NDCGs_eva3.at[algorithm, pop_n+"_med"] = ndcg_df.loc[algorithm]['med']
                books_NDCGs_eva3.at[algorithm, pop_n+"_high"] =ndcg_df.loc[algorithm]['high']

In [ ]:
with open("latex_tables/books_NDCGs_eva1.tex", "w") as tf:
    tf.write(np.round(books_NDCGs_eva1,3).to_latex())
with open("latex_tables/books_NDCGs_eva2.tex", "w") as tf:
    tf.write(np.round(books_NDCGs_eva2,3).to_latex())
with open("latex_tables/books_NDCGs_eva3.tex", "w") as tf:
    tf.write(np.round(books_NDCGs_eva3,3).to_latex())

In [ ]:
new_results_location = 'results/'
for combi in all_combinations:
    
    if 'books' in combi: 
        eva_str = combi[1]
        pop_n = combi[2]
        print(eva_str, pop_n)
        print('----------------')
        path_ttest = new_results_location+"NDCG_ttests_books_"+eva_str+"_"+pop_n+".csv"
        pvalues = pd.read_csv(path_ttest, index_col=0).drop(['Random','MostPop'])
        print((pvalues<0.005)&(pvalues>0.0)&(pvalues!=1.0))
        print(pvalues)

### Movies

In [ ]:
movies_NDCGs_eva1 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
movies_NDCGs_eva2 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
movies_NDCGs_eva3 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)

In [ ]:
for combi in all_combinations:
    if "movies" in combi:
        eva_str = combi[1]
        pop_n = combi[2]
        #print(eva_str, pop_n)
        path_ttest = new_results_location+"NDCGs_movies_"+eva_str+"_"+pop_n+".csv"
        ndcg_df = pd.read_csv(path_ttest, index_col=0)
        #print(ndcg_df)
        
        if eva_str=="eva_one":
            for algorithm in algo_names:
                movies_NDCGs_eva1.at[algorithm, pop_n+"_low"] = ndcg_df.loc[algorithm]['low']
                movies_NDCGs_eva1.at[algorithm, pop_n+"_med"] = ndcg_df.loc[algorithm]['med']
                movies_NDCGs_eva1.at[algorithm, pop_n+"_high"] =ndcg_df.loc[algorithm]['high']
                        
        elif eva_str=="eva_two":
            for algorithm in algo_names:
                movies_NDCGs_eva2.at[algorithm, pop_n+"_low"] = ndcg_df.loc[algorithm]['low']
                movies_NDCGs_eva2.at[algorithm, pop_n+"_med"] = ndcg_df.loc[algorithm]['med']
                movies_NDCGs_eva2.at[algorithm, pop_n+"_high"] =ndcg_df.loc[algorithm]['high']
            
        else:
            for algorithm in algo_names:
                movies_NDCGs_eva3.at[algorithm, pop_n+"_low"] = ndcg_df.loc[algorithm]['low']
                movies_NDCGs_eva3.at[algorithm, pop_n+"_med"] = ndcg_df.loc[algorithm]['med']
                movies_NDCGs_eva3.at[algorithm, pop_n+"_high"] =ndcg_df.loc[algorithm]['high']

In [ ]:
with open("latex_tables/movies_NDCGs_eva1.tex", "w") as tf:
    tf.write(np.round(movies_NDCGs_eva1,3).to_latex())
with open("latex_tables/movies_NDCGs_eva2.tex", "w") as tf:
    tf.write(np.round(movies_NDCGs_eva2,3).to_latex())
with open("latex_tables/movies_NDCGs_eva3.tex", "w") as tf:
    tf.write(np.round(movies_NDCGs_eva3,3).to_latex())

In [ ]:
new_results_location = 'results/'
for combi in all_combinations:
    
    if 'movies' in combi: 
        eva_str = combi[1]
        pop_n = combi[2]
        print(eva_str, pop_n)
        print('----------------')
        path_ttest = new_results_location+"NDCG_ttests_movies_"+eva_str+"_"+pop_n+".csv"
        pvalues = pd.read_csv(path_ttest, index_col=0).drop(['MostPop'])
        print((pvalues<0.005)&(pvalues>0.0)&(pvalues!=1.0))
        print(pvalues)